In [11]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import random


In [12]:
if torch.cuda.is_available():
    DTYPE = torch.cuda.FloatTensor
else:
    DTYPE = torch.FloatTensor
    

In [13]:
data = open('texts/Lovecraft.txt','r').read().lower()
alphabet = set(data)

ix_to_char = {k:v for k,v in enumerate(alphabet)}
char_to_ix = {k:v for v,k in enumerate(alphabet)}

In [14]:
# Define function to prepare sequences

def prepare_seq(data, drop_last=False):
    
    sequences = []
    
    # Create (len(data)/SEQ_LEN+1) number of vectors of SEQ_LEN lenght
    for i in range(0,len(data),SEQ_LEN+1):

        chars = [char_to_ix[c] for c in data[i:i+SEQ_LEN+1]]
        sequences.append(chars)
    
    # Drop last batch if incomplete`
    if drop_last and len(sequences) % BATCH_SIZE != 0:
        
        index = len(sequences)//BATCH_SIZE * BATCH_SIZE
        del(sequences[index:])
    
    
    # Drop last sequence if incomplete
    elif len(sequences[-1]) != SEQ_LEN:
        del(sequences[-1])
    
    sequences = np.array([sequences]).reshape((-1,SEQ_LEN+1))
    
    # Create inputs and targets
    inputs = sequences[:,:-1]
    targets = sequences[:,1:]
    
    
    # Convert sequences to variables
    #inputs = autograd.Variable(torch.Tensor(inputs).type(DTYPE))
    #targets = autograd.Variable(torch.Tensor(targets).type(DTYPE))    
    
    
    return inputs,targets


In [36]:
class LSTM(nn.Module):

    def __init__(self,alphabet_size, hidden_dim, embedding_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(alphabet_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,hidden_dim,NUM_LAYERS)
        self.h2O = nn.Linear(hidden_dim, alphabet_size)
        
        self.hidden = self.init_hidden()
        
        
    def init_hidden(self):
       
        return (autograd.Variable(torch.zeros(NUM_LAYERS, BATCH_SIZE, self.hidden_dim).type(DTYPE)),
                autograd.Variable(torch.zeros(NUM_LAYERS, BATCH_SIZE, self.hidden_dim).type(DTYPE)))

    def forward(self, sequence):
        embeds = self.embedding(sequence)
        #print(embeds.view(SEQ_LEN,BATCH_SIZE,-1))
        lstm_out, self.hidden = self.lstm(embeds.view(SEQ_LEN,BATCH_SIZE,-1),self.hidden)
        #print(lstm_out)
        out = self.h2O(lstm_out.view(-1,self.hidden_dim))
        return out


In [37]:
NUM_LAYERS = 1
BATCH_SIZE = 128
HIDDEN_DIM = 64
EMBEDDING_DIM = 32
SEQ_LEN = 64


In [38]:
rnn = LSTM(len(alphabet),HIDDEN_DIM,EMBEDDING_DIM).type(DTYPE)
optimizer = optim.Adam(rnn.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()

epochs = 100

In [18]:
inputs,targets = prepare_seq(data,drop_last=True)

In [48]:
def train(data):
    
    inputs,targets = prepare_seq(data,drop_last=True)
    
    rnn.train(True)
    for epoch in range(epochs):

        losses = np.array([])
        
        for i in range(0,inputs.shape[0],BATCH_SIZE):

            rnn.zero_grad()
            rnn.hidden = rnn.init_hidden()
            
            batch = torch.Tensor(inputs[i:i+BATCH_SIZE]).long()
            
            out = rnn(autograd.Variable(batch))
                        
            target = autograd.Variable(torch.Tensor(targets[i:i+BATCH_SIZE]).long())
            
            loss = criterion(out,target.view(-1))
            losses = np.append(losses,loss.data[0])


            loss.backward()
            optimizer.step()


        print("Epoch {}/{}\nLoss: {:.2f}".format(epoch+1,epochs,losses.mean()))
        print("="*15)
    
    
    rnn.zero_grad()


In [49]:
chunks = 500000

#for i in range(0,len(data),chunks):
train(data)

KeyboardInterrupt: 

In [ ]:
string = ''  
temperature = 0.5
inputs,targets = prepare_seq(data[:10000],drop_last=True)

#idxs = permutation[i:i+BATCH_SIZE]
rnn.train(False)
for i in range(0,inputs.size()[0],BATCH_SIZE):
    
    out = rnn(inputs[i:i+BATCH_SIZE].view(SEQ_LEN,BATCH_SIZE,-1))
    _ ,ix = out.topk(5)
    
    for j in range(ix.shape[0]):
    
        t = random.random()

        if t < temperature:
            selection = random.randint(1,ix.shape[1]-1)
        else:
            selection = 0
        
        string += ix_to_char[ix[j,selection].data[0]]

print(string)       
#print(string,file=open('texts/output.txt','w'))
    
    #with open('texts/output.txt', mode='wt', encoding='utf-8') as myfile:
        #myfile.writelines(' '.join(lyrics))


In [ ]:
# One hot encoder for chars or embedding
# temperature softmax
# install cudnn
# overwrite dataloader